In [9]:
import pandas as pd
import os
import replicaEVSE.datautils as datautils

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

mode = 'PRIVATE_AUTO'
test = False
datadir = '../../data/'

In [5]:
#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv(datadir+'EIA_demand_summary.csv') 
if test:
    # df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))
    df = df.head(10000)
    df = df.loc[df['mode'] == mode]
    simulation_id = 'dev'

else: 
    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]
    # take out the mobile and commercial MHDV

In [7]:
person_list = df['person_id'].unique()

In [13]:
person_df_list = []
for person in person_list[:3]:
    for day in ['thursday', 'saturday']:
        print(person, day)
        person_day_cond = (df['person_id'] == person) & (df['weekday'] == day)
        person_df_sub = df.loc[person_day_cond]
        if len(person_df_sub) == 0:
            print('no data for person {} on {}'.format(person, day))
            continue
        # calc the dwell time
        person_df_dwelltime = datautils.calculate_stop_duration(person_df_sub)
        person_df_list.append(person_df_dwelltime)
    

10000002909730821040 thursday
10000002909730821040 saturday
10000003183049770926 thursday
10000003183049770926 saturday
10000005160936665771 thursday
no data for person 10000005160936665771 on thursday
10000005160936665771 saturday


In [14]:
new_df = pd.concat(person_df_list)

In [15]:
new_df[['person_id', 'start_time', 'end_time', 'stop_duration', 'weekday']]

,person_id,start_time,end_time,stop_duration,weekday
0,10000002909730821040,0 days 08:20:42,0 days 08:39:00,0 days 00:33:45,thursday
1,10000002909730821040,0 days 09:12:45,0 days 09:19:56,0 days 04:09:34,thursday
7,10000002909730821040,0 days 13:29:30,0 days 13:39:45,0 days 00:58:40,thursday
8,10000002909730821040,0 days 14:38:25,0 days 14:48:33,0 days 01:07:11,thursday
9,10000002909730821040,0 days 15:55:44,0 days 16:10:30,0 days 05:57:23,thursday
11,10000002909730821040,0 days 22:07:53,0 days 22:16:46,0 days 00:22:28,thursday
12,10000002909730821040,0 days 22:39:14,0 days 23:06:23,0 days 09:14:19,thursday
6,10000002909730821040,0 days 13:28:23,0 days 13:51:27,0 days 07:12:54,saturday
10,10000002909730821040,0 days 21:04:21,0 days 21:38:18,0 days 15:50:05,saturday
17,10000003183049770926,0 days 19:29:21,0 days 19:45:47,0 days 01:00:10,thursday


In [21]:
groupby_df_stop_dur = df.head(23).groupby(['person_id', 'weekday']).apply(datautils.calculate_stop_duration)

In [22]:
groupby_df_stop_dur[['person_id', 'start_time', 'end_time', 'stop_duration', 'weekday']]

person_id      start_time   
person_id            weekday                                             
10000002909730821040 saturday 6   10000002909730821040 0 days 13:28:23  \
                              10  10000002909730821040 0 days 21:04:21   
                     thursday 0   10000002909730821040 0 days 08:20:42   
                              1   10000002909730821040 0 days 09:12:45   
                              7   10000002909730821040 0 days 13:29:30   
                              8   10000002909730821040 0 days 14:38:25   
                              9   10000002909730821040 0 days 15:55:44   
                              11  10000002909730821040 0 days 22:07:53   
                              12  10000002909730821040 0 days 22:39:14   
10000003183049770926 saturday 14  10000003183049770926 0 days 11:35:32   
                              15  10000003183049770926 0 days 13:20:08   
                     thursday 17  10000003183049770926 0 days 19:29:21   
                              18  10000003183049770926 0 days 20:45:57   
10000005160936665771 saturday 23  10000005160936665771 0 days 17:47:00   
10000013294719422231 thursday 30  10000013294719422231 0 days 09:28:46   
                              31  10000013294719422231 0 days 10:25:33   
10000015626584625931 saturday 39  10000015626584625931 0 days 09:04:05   
                              40  10000015626584625931 0 days 13:28:58   
                              44  10000015626584625931 0 days 18:11:43   
                              45  10000015626584625931 0 days 18:37:16   
                     thursday 41  10000015626584625931 0 days 14:19:41   
                              42  10000015626584625931 0 days 15:43:13   
                              43  10000015626584625931 0 days 15:58:25   

                                        end_time    stop_duration   weekday  
person_id            weekday                                                 
10000002909730821040 saturday 6  0 days 13:51:27  0 days 07:12:54  saturday  
                              10 0 days 21:38:18  0 days 15:50:05  saturday  
                     thursday 0  0 days 08:39:00  0 days 00:33:45  thursday  
                              1  0 days 09:19:56  0 days 04:09:34  thursday  
                              7  0 days 13:39:45  0 days 00:58:40  thursday  
                              8  0 days 14:48:33  0 days 01:07:11  thursday  
                              9  0 days 16:10:30  0 days 05:57:23  thursday  
                              11 0 days 22:16:46  0 days 00:22:28  thursday  
                              12 0 days 23:06:23  0 days 09:14:19  thursday  
10000003183049770926 saturday 14 0 days 11:44:09  0 days 01:35:59  saturday  
                              15 0 days 13:26:13  0 days 22:09:19  saturday  
                     thursday 17 0 days 19:45:47  0 days 01:00:10  thursday  
                              18 0 days 21:01:01  0 days 22:28:20  thursday  
10000005160936665771 saturday 23 0 days 19:23:02  0 days 22:23:58  saturday  
10000013294719422231 thursday 30 0 days 09:31:14  0 days 00:54:19  thursday  
                              31 0 days 10:28:01  0 days 23:00:45  thursday  
10000015626584625931 saturday 39 0 days 09:12:00  0 days 04:16:58  saturday  
                              40 0 days 13:38:18  0 days 04:33:25  saturday  
                              44 0 days 18:25:43  0 days 00:11:33  saturday  
                              45 0 days 18:50:39  0 days 14:13:26  saturday  
                     thursday 41 0 days 14:37:29  0 days 01:05:44  thursday  
                              42 0 days 15:49:22  0 days 00:09:03  thursday  
                              43 0 days 16:07:07  0 days 22:12:34  thursday

In [ ]:
groupby_df_stop_dur.reset_index()

In [ ]:
groupby_df_stop_dur = groupby_df_stop_dur.reset_index()
# groupby_df_stop_dur.to_parquet(datadir+'wa_ldv_trips_with_county_dwell_time.parquet')